<a href="https://colab.research.google.com/github/ak-ghoul/Deep-Learning/blob/master/TCS_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blueprint of Recommender System

###Step by step procedure

In [ ]:
# importing libraries
import pandas as pd
import numpy as np

In [ ]:
#uploading dataset I took from Kaggle containig Job Profiles
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Combined_Jobs_Final.csv to Combined_Jobs_Final.csv
User uploaded file "Combined_Jobs_Final.csv" with length 28046958 bytes


In [ ]:
job_details = pd.read_csv('Combined_Jobs_Final.csv', engine='python', error_bad_lines=False)
print(job_details.shape)
job_details.head()

(14999, 23)


,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,Address,Latitude,Longitude,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,NaN,37.443346,-122.161170,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.00,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,NaN,37.789830,-122.404268,Food and Beverages,\n\nNew French Brasserie in S.F. Financial Di...,NaN,0.00,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,NaN,37.795597,-122.402963,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.00,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC
3,121,1,open,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,California,CA,NaN,37.685073,-122.400275,Food and Beverages,? Serve food/drinks to customers in a profess...,NaN,10.55,NaN,NaN,Part-Time,NaN,2013-09-04 15:40:30 UTC,2014-08-16 15:35:38 UTC
4,127,1,open,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,California,CA,NaN,34.073384,-118.460439,Food and Beverages,"Located at the heart of Hollywood, we are one ...",NaN,10.55,NaN,NaN,Part-Time,NaN,2013-07-17 15:26:18 UTC,2014-08-16 15:35:40 UTC


#Data Cleaning

In [ ]:
# Data Cleaning 
columns_keep = ['Job.ID', 'Slug', 'Title', 'Position', 'Company', 'City', 'Employment.Type', 'Education.Required', 'Job.Description']
job_details = job_details[columns_keep]
job_details.head() 

,Job.ID,Slug,Title,Position,Company,City,Employment.Type,Education.Required,Job.Description
0,111,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,Part-Time,NaN,Tacolicious' first Palo Alto store just opened...
1,113,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,Part-Time,NaN,\n\nNew French Brasserie in S.F. Financial Di...
2,117,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,Part-Time,NaN,We are a popular Mediterranean wine bar and re...
3,121,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,Part-Time,NaN,? Serve food/drinks to customers in a profess...
4,127,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,Part-Time,NaN,"Located at the heart of Hollywood, we are one ..."


In [ ]:
job_details.isnull().sum()   # checking how many null values are present where

Job.ID                  0
Slug                    0
Title                   0
Position                0
Company               600
City                   11
Employment.Type        10
Education.Required    100
Job.Description         7
dtype: int64

In [ ]:
# As we can see, some columns have Not Assigned values.
# Dropping the rows with Not Assigned 'Company, City & Job.Description'
job_details.dropna(subset = ["Company"], inplace=True)
job_details.dropna(subset = ["City"], inplace=True)
job_details.dropna(subset = ["Job.Description"], inplace=True)

#Few rows have Education not defined.
#Replacing the NaN values of Education.Required type with "Not Specified" and Employment.Required as "Full-Time/Part-Time"
job_details['Education.Required'] = job_details['Education.Required'].fillna("Not Specified")
job_details['Employment.Type'] = job_details['Employment.Type'].fillna("Full-Time/Part-Time")
job_details.isnull().sum()

Job.ID                0
Slug                  0
Title                 0
Position              0
Company               0
City                  0
Employment.Type       0
Education.Required    0
Job.Description       0
dtype: int64

## Corpus
#### A collection of text in readable format used in NLP.
####The details of job in dataset will be combined to form a readable corpus

In [ ]:
#Corpus is made to apply NLP easily
final_jobs = job_details['Job.ID']
final_jobs = pd.DataFrame(final_jobs)  # need to make it pandas dataframe before adding pandas columns into it.
final_jobs['pos_company_city_emptype_jobdesc'] = job_details["Position"]+" "+job_details["Company"]+" "+job_details["City"]+" "+job_details["Employment.Type"]+" "+job_details["Job.Description"]

In [ ]:
# cleaning the corpus 
# replacing unnecessary symbols with space
final_jobs['pos_company_city_emptype_jobdesc'] = final_jobs['pos_company_city_emptype_jobdesc'].str.replace('[^a-zA-Z \n\.]'," ")
# converting all characters to lowercase
final_jobs['pos_company_city_emptype_jobdesc'] = final_jobs['pos_company_city_emptype_jobdesc'].str.lower()
final_jobs.head()

,Job.ID,pos_company_city_emptype_jobdesc
0,111,server tacolicious palo alto part time tacolic...
1,113,kitchen staff chef claude lane san francisco p...
2,117,bartender machka restaurants corp. san francis...
3,121,server teriyaki house brisbane part time se...
4,127,kitchen staff chef rosa mexicano sunset los ...


#NLPTK(Natural Language Processing Toolkit)
### --> Stop Words and Porter Stemming
##### Stop word means removing unnecessary words like a, an, the, better, like, etc.
##### Stemming is removing duplicate words with their root form of word.

In [ ]:
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# applying stop words
stop = stopwords.words('english')
final_jobs['pos_company_city_emptype_jobdesc'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

# applying stemmer - This basically removes all duplicate words or words having same meaning. It gives root forms of words. 
stemmer = PorterStemmer()
final_jobs['pos_company_city_emptype_jobdesc'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x))
final_jobs.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Job.ID,pos_company_city_emptype_jobdesc
0,111,server tacolicious palo alto part time tacolic...
1,113,kitchen staff chef claude lane san francisco p...
2,117,bartender machka restaurants corp. san francis...
3,121,server teriyaki house brisbane part time se...
4,127,kitchen staff chef rosa mexicano sunset los ...


## Dealing with data of Users who applied for Jobs
##### We have 3 different datasets of users, 
##### -> Experience
##### -> Jobs Viewed
##### -> Positions of Interest
##### I will combine these datasets and  use it in Cosine Similarity for recommending these users to job posts.

In [ ]:
#Uploading the Experience Dataset
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Experience.csv to Experience.csv
User uploaded file "Experience.csv" with length 2836104 bytes


In [ ]:
experience = pd.read_csv('Experience.csv')
experience.head()

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
2,10001,Machine Operator,comptec inc,Custer,Washington,WA,1997-01-01,1999-01-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
3,10003,maintenance technician,Winn residental,washington,District of Columbia,DC,NaN,NaN,"Necessary maintenance for ""Make Ready"" Plumbin...",10.0,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC
4,10003,Electrical Helper,michael and son services,alexandria,Virginia,VA,NaN,NaN,repair and services of electrical construction,NaN,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC


In [ ]:
# making this dataset ready for use. Repeating the steps of data cleaning
experience_final = experience['Applicant.ID']
experience_final = pd.DataFrame(experience_final)
experience_final['pos_empname'] = experience['Position.Name'] +" "+ experience['Employer.Name'] # these are the only details we require
# removing null rows
experience_final.dropna()

experience_final['pos_empname'] = experience_final['pos_empname'].str.replace('[^a-zA-Z \n\.]'," ")
# converting all characters to lowercase
experience_final['pos_empname'] = experience_final['pos_empname'].str.lower()
experience_final = experience_final.sort_values(by='Applicant.ID')
experience_final.head()

,Applicant.ID,pos_empname
2763,2,volunteer school for self healing
2762,2,writer for the uloop blog cecilia abate
3759,3,marketing intern honda
3758,3,server aloha beach resort
3757,3,prep cook moscone center


In [ ]:
# As we can see, same id have more than one experience, so clubbing them together..
experience_final = experience_final.groupby('Applicant.ID', sort=False).sum().reset_index()
experience_final.head()

,Applicant.ID,pos_empname
0,2,volunteer school for self healingwriter for th...
1,3,marketing intern hondaserver aloha beach resor...
2,6,project assistant iom
3,8,deli clerk server cashier food prep order t...
4,11,cashier cristina green


In [ ]:
# Uploading Jobs viewed by candidates Dataset
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Job_Views.csv to Job_Views.csv
User uploaded file "Job_Views.csv" with length 3291093 bytes


In [ ]:
job_views = pd.read_csv('Job_Views.csv')
job_views.head()

,Applicant.ID,Job.ID,Title,Position,Company,City,State.Name,State.Code,Industry,View.Start,View.End,View.Duration,Created.At,Updated.At
0,10000,73666,Cashiers & Valets Needed! @ WallyPark,Cashiers & Valets Needed!,WallyPark,Newark,New Jersey,NJ,NaN,2014-12-12 20:12:35 UTC,2014-12-12 20:31:24 UTC,1129.0,2014-12-12 20:12:35 UTC,2014-12-12 20:12:35 UTC
1,10000,96655,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's,Garden City,New York,NY,NaN,2014-12-12 20:08:50 UTC,2014-12-12 20:10:15 UTC,84.0,2014-12-12 20:08:50 UTC,2014-12-12 20:08:50 UTC
2,10001,84141,Part Time Showroom Sales / Cashier @ Grizzly I...,Part Time Showroom Sales / Cashier,Grizzly Industrial Inc.,Bellingham,Washington,WA,NaN,2014-12-12 20:12:32 UTC,2014-12-12 20:17:18 UTC,286.0,2014-12-12 20:12:32 UTC,2014-12-12 20:12:32 UTC
3,10002,77989,Event Specialist Part Time @ Advantage Sales &...,Event Specialist Part Time,Advantage Sales & Marketing,Simpsonville,South Carolina,SC,NaN,2014-12-12 20:39:23 UTC,2014-12-12 20:42:13 UTC,170.0,2014-12-12 20:39:23 UTC,2014-12-12 20:39:23 UTC
4,10002,69568,Bonefish - Kitchen Staff @ Bonefish Grill,Bonefish - Kitchen Staff,Bonefish Grill,Greenville,South Carolina,SC,NaN,2014-12-12 20:43:25 UTC,2014-12-12 20:43:58 UTC,33.0,2014-12-12 20:43:25 UTC,2014-12-12 20:43:25 UTC


In [ ]:
#Extracting wanted columns
job_views_final = job_views['Applicant.ID']
job_views_final = pd.DataFrame(job_views_final)
job_views_final['pos_com_city'] = job_views['Position'] +" "+ job_views['Company'] +" "+ job_views['City']

#Cleaning up & converting to lowercase
job_views_final['pos_com_city'] = job_views_final['pos_com_city'].str.replace("[^a-zA-Z \n\.]", " ")
job_views_final['pos_com_city'] = job_views_final['pos_com_city'].str.lower()
job_views_final.sort_values(by="Applicant.ID")
job_views_final.head()

,Applicant.ID,pos_com_city
0,10000,cashiers valets needed wallypark newark
1,10000,macy s seasonal retail fragrance cashier ga...
2,10001,part time showroom sales cashier grizzly ind...
3,10002,event specialist part time advantage sales m...
4,10002,bonefish kitchen staff bonefish grill greenv...


In [ ]:
# opening Positions of Interest Dataset
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Positions_Of_Interest.csv to Positions_Of_Interest.csv
User uploaded file "Positions_Of_Interest.csv" with length 478462 bytes


In [ ]:
posof_interest = pd.read_csv("Positions_Of_Interest.csv")
posof_interest.head()

,Applicant.ID,Position.Of.Interest,Created.At,Updated.At
0,10003,security officer,2014-12-12 21:20:54 UTC,2014-12-12 21:20:54 UTC
1,10007,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
2,10007,Bartender,2014-08-14 15:56:44 UTC,2015-02-19 23:21:28 UTC
3,10008,Host,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
4,10008,Barista,2014-08-14 15:56:43 UTC,2015-02-18 02:35:06 UTC


In [ ]:
posof_interest = posof_interest.drop("Created.At", axis=1)
posof_interest = posof_interest.drop("Updated.At", axis=1)

#cleaning and converting to lower
posof_interest['Position.Of.Interest'] = posof_interest['Position.Of.Interest'].str.replace("[^a-zA-Z \n\.]", " ")
posof_interest['Position.Of.Interest'] = posof_interest['Position.Of.Interest'].str.lower()
posof_interest.head()

,Applicant.ID,Position.Of.Interest
0,10003,security officer
1,10007,server
2,10007,bartender
3,10008,host
4,10008,barista


### Merging the Datasets to make one Ultimate Dataset of Applicants
##### This will be used for comparing with the Job Details Dataset

In [ ]:
# combining Experience & Job Views Datasets
outer_join = pd.merge(experience_final, job_views_final, on="Applicant.ID", how='outer')
outer_join = outer_join.sort_values(by="Applicant.ID")
outer_join = outer_join.fillna(' ')
print(outer_join.shape)
outer_join.head()

(15383, 3)


,Applicant.ID,pos_empname,pos_com_city
0,2,volunteer school for self healingwriter for th...,
1,3,marketing intern hondaserver aloha beach resor...,
2,6,project assistant iom,
3,8,deli clerk server cashier food prep order t...,
4,11,cashier cristina green,


In [ ]:
# Combining the resultant dataset with Positions Of Interest Dataset
candidate = pd.merge(outer_join, posof_interest, on = 'Applicant.ID', how= 'outer')
candidate = candidate.sort_values(by = "Applicant.ID")
candidate = candidate.fillna(' ')
candidate.head()

,Applicant.ID,pos_empname,pos_com_city,Position.Of.Interest
0,2,volunteer school for self healingwriter for th...,,
1,3,marketing intern hondaserver aloha beach resor...,,
2,6,project assistant iom,,
3,8,deli clerk server cashier food prep order t...,,
4,11,cashier cristina green,,


In [ ]:
# Combining all these columns to make one single column just like it is present in Jobs Dataset
applicant_final = candidate["Applicant.ID"]
applicant_final = pd.DataFrame(applicant_final)
applicant_final["experience_jobview_posofinterest"] = candidate["pos_empname"].map(str)+" "+candidate["pos_com_city"]+" "+candidate["Position.Of.Interest"]
applicant_final.head()

,Applicant.ID,experience_jobview_posofinterest
0,2,volunteer school for self healingwriter for th...
1,3,marketing intern hondaserver aloha beach resor...
2,6,project assistant iom
3,8,deli clerk server cashier food prep order t...
4,11,cashier cristina green


# TF-IDF (Term Frequency - Inverse Document Frequency)

##### TF - The count of how many times a word has occured in the document.
##### IDF - The importance of a word, more it occurs, less is it's importance.

###### I will use the tf-idf vectorizer from Scikit Learn library.

###### I will apply tf-idf on Final Applicants Dataset and on one job id because applicants will be recommended for this one job id.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer    #importing tf-idf vectorizer
tf_idf = TfidfVectorizer(use_idf = True)

tf_idf_applicants = tf_idf.fit_transform(applicant_final['experience_jobview_posofinterest'])
# pandas_tfidf_applicants = pd.DataFrame(tf_idf_applicants[0].T.todense(), index=tf_idf.get_feature_names(), columns=["Tf-idf"])
# pandas_tfidf_applicants = pandas_tfidf_applicants.sort_values("Tf-idf", ascending=False)
# pandas_tfidf_applicants.head(20)

In [ ]:
# taking any one job ID
job_id = 111
one_job = final_jobs[final_jobs["Job.ID"] == job_id]
one_job

,Job.ID,pos_company_city_emptype_jobdesc
0,111,server tacolicious palo alto part time tacolic...


In [ ]:
tf_idf_job = tf_idf.transform(one_job['pos_company_city_emptype_jobdesc'])
print(tf_idf_job)

  (0, 14235)	0.35686175741777343
  (0, 14108)	0.13951182973299198
  (0, 13984)	0.19942225032250227
  (0, 13838)	0.17548549548486653
  (0, 13413)	0.13204348077902364
  (0, 13079)	0.17548549548486653
  (0, 12980)	0.11344362259846906
  (0, 12940)	0.05345950020161394
  (0, 12858)	0.13660873010047558
  (0, 12610)	0.28160088348191686
  (0, 12316)	0.09921681840112731
  (0, 12158)	0.10439695399346109
  (0, 12046)	0.1458579673361562
  (0, 11570)	0.06967196563562301
  (0, 11569)	0.1861781694031969
  (0, 10874)	0.10433095334108843
  (0, 10307)	0.11013771603977987
  (0, 9672)	0.05380277978446565
  (0, 9603)	0.22862455980469545
  (0, 9497)	0.17548549548486653
  (0, 9030)	0.07708389413364936
  (0, 8974)	0.10598151883817078
  (0, 7843)	0.3170632852352687
  (0, 7020)	0.19942225032250227
  (0, 6411)	0.08996730377214683
  (0, 6350)	0.19167495962819206
  (0, 6023)	0.09649601492490693
  (0, 4931)	0.10293379380620389
  (0, 4816)	0.12320405456235023
  (0, 3801)	0.13869863595097065
  (0, 2565)	0.129421955644

# Cosine Similarity

##### I have tf-idf vectors of both things, applicant dataset & a Job. 
##### I will apply Cosine Similarity on both of them to finally find the similarity and recommend applicants to a recruiter for a job.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Applying Cosine-Sine similarity
output = list(map(lambda x: cosine_similarity(tf_idf_job, x), tf_idf_applicants))
# print(output[0])

#getting applicant's id
output_order = sorted(range(len(output)), key=lambda x: output[x].all(), reverse=True)[:30]
count = 0
recommendation = pd.DataFrame(columns = ["Job.ID", "Recommended Applicants ID"])
for i in output_order:
    recommendation.at[count, 'Job.ID'] = job_id
    recommendation.at[count,'Recommended Applicants ID'] = applicant_final['Applicant.ID'][i]
    count += 1

In [ ]:
recommendation    # Hence, Applicants are recommended to a given Job position

,Job.ID,Recommended Applicants ID
0,111,2
1,111,8
2,111,11
3,111,12
4,111,13
5,111,18
6,111,21
7,111,22
8,111,23
9,111,24
